In [ ]:
%matplotlib notebook
from ipywidgets import interact, SelectionSlider, IntSlider, FloatSlider
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np

from IPython.display import YouTubeVideo, HTML, Audio
from bokeh.layouts import column, row, gridplot
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, show, output_notebook
output_notebook()

# Fuga espectral y técnica de enventanado

## Definiciones

- Diremos que una función es limitada en el dominio del tiempo si

$$
s(t) = 0 \quad \forall |t| > T,
$$

para alguna constante $T$

- Diremos que una función es limitada en el dominio de la frecuencia o de ancho de banda limitado si

$$
S(\omega) = 0 \quad \forall |\omega| > \Omega,
$$

para alguna constante $\Omega$

## Transformada de Fourier de una señal limitada en el tiempo

Como vimos en la lección anterior, el espectro de 

$$
s(t) = \cos(2\pi f_0 t)
$$

es

$$
S(f) = \frac{1}{2} \left(\delta(f-f_0) + \delta(f+f_0) \right)
$$

Pero esto asume que $s(t)$ existen para todo $t$. 

<div class="alert alert-info">
    
En la práctica siempre trabajamos con señales de duración finita

</div>

Una versión "finita" y de duración $T>0$ de la señal anterior es equivalente a 

$$
s_T(t) = \cos(2 \pi f_0 t) \cdot \text{rect}(t/T),
$$

donde 

$$
\text{rect}(x) = \begin{cases} 1 & |x| \leq 1 \\ 0 & |x| > 0 \end{cases}
$$

se llama **ventana rectangular** o pulso cuadrado



In [ ]:
T = 5 # Duración de la ventana
Fs = 20 # Frecuencia de muestre
f0 = 1.2345 # Frecuencia fundamental
t = np.arange(-10, 10, step=1/Fs)
s = np.cos(2*np.pi*f0*t)
w = np.zeros_like(t)
w[np.absolute(t) < T/2] = 1
sT = s*w

In [ ]:
p1 = Figure(plot_width=600, plot_height=280, toolbar_location="below", x_range=(-10, 10))
p1.line(t, s,  color='green', alpha=0.75, line_width=4, legend_label='Señal original')
p1.line(t, sT, line_width=3, alpha=0.75, legend_label='Señal recortada')
p1.line(t, w, line_width=3, alpha=0.75, color='black', line_dash='dashed', legend_label='Ventana')
p1.xaxis[0].axis_label = 'Tiempo [s]'
show(p1)

### Transformada de Fourier de la ventana rectangular

En la lección 2 vimos que la transformada de Fourier del pulso cuadrado es

$$
S(f) = \frac{T}{\pi f T} \sin(\pi f T) = T \text{sinc}(f T)
$$

donde la función sinc se define como

$$
\text{sinc}(x) = \frac{1}{\pi x} \sin(\pi x) 
$$

Puedes revisar la demostración de esta transformada [aquí](http://www.thefouriertransform.com/pairs/box.php)


El siguiente gráfico muestra como cambia $S(f)$ para distintos valores de $T$

In [ ]:
f = np.arange(-2, 2, step=1e-2)
S = {}
Ts = [2, 5, 10]
for T in Ts:
    S[T] = T*np.sinc(f*T)

In [ ]:
from bokeh.palettes import Dark2_5 as palette
p1 = Figure(plot_width=600, plot_height=280, toolbar_location="below")
for T, color in zip(Ts, palette):
    p1.line(f, S[T], line_width=3, alpha=0.75, legend_label=f"T={T}", color=color)
p1.xaxis[0].axis_label = 'Frecuencia [Hz]'
show(p1)

<div class="alert alert-info">
    
Mientras mayor sea la duración temporal $T$ de la ventana más angosto y concentrado será su espectro

</div>

### Transformada de Fourier de $s_T(t)$

Usando la propiedad de modulación de la transformada de Fourier tenemos que

$$
\begin{align}
S_T(f) &= \mathbb{FT}[s_T(t)] \nonumber \\
&= \mathbb{FT}[\text{rect}_T(t) \cdot s(t) ] \nonumber \\
&=  \mathbb{FT}[\text{rect}_T(t)] * \mathbb{FT}[s_T(t)]    \nonumber \\
&=  T \text{sinc}(f T) *  \frac{1}{2} \left(\delta(f-f_0) + \delta(f+f_0) \right) \nonumber 
\end{align}
$$

<div class="alert alert-info">
    
Multiplicar por una ventana rectangular en el tiempo es equivalente a convolucionar con un sinc en frecuencia

</div>


En la lección anterior vimos lo que significa convolucionar con un impulso, en este caso el resultado es

$$
S_T(f) = \frac{T}{2} \left [ \text{sinc}((f - f_0)T) +  \text{sinc}((f + f_0)T) \right]
$$

En lugar de un impulso lo que observamos es un $\text{sinc}$ ubicado en la frecuencia fundamental, como muestra la siguiente animación

In [ ]:
%%capture
fig, ax = plt.subplots(2, figsize=(7, 4), sharex=True, tight_layout=True)
f = np.arange(-4, 4, step=1e-3)

def FT_rect(f, T=5):
    return T*np.sinc(T*(f))

def FT_cos(f):
    S = np.zeros_like(f)
    S[2000] = 1
    S[6000] = 1
    return S
    
conv_s = np.convolve(FT_rect(f), FT_cos(f), mode='same')

def update(frame = 0): 
    loc = 0.1*frame - 4
    ax[0].cla(); ax[1].cla()
    p1, p2 = FT_rect(f-loc), FT_cos(f)
    ax[0].plot(f, p1, label='espectro de rect'); 
    ax[0].plot(f, p2, label='espectro de coseno'); 
    ax[0].legend()
    ax[1].plot(f, conv_s); 
    ax[1].set_xlabel('Frecuencia [Hz]'); 
    ax[1].scatter(loc, np.sum(p1*p2), s=100, c='k')
    return ()
    
anim = animation.FuncAnimation(fig, update, frames=80, interval=100, blit=True)

In [ ]:
HTML(anim.to_html5_video())

### Espectro de amplitud en función de $T$

Mientras más pequeño es $T$ 

- más "corta" es $s_T(t)$ 
- más anchos son los "lóbulos" de $S_T(f)$ 


<img src="../images/mainsidelobe.png" width="300">

- El lóbulo principal (*mainlobe* en la figura) es el que está relacionado a información real de la señal
- Los lóbulos laterales (*sidelobe* en la figura) son un efecto de la convolución con el **sinc**
- La proporción entre el tamaño del lóbulo principal y los lóbulos laterales aumenta con $T$


La siguiente figura muestra la señal coseno truncada con una ventana rectangular (columna izquierda) y su correspondiente espectro de amplitud (columna derecha)

In [ ]:
import scipy.fft as sfft

Ts = [2, 5, 10, 15] # Duración de la ventana
Fs = 20 # Frecuencia de muestre
f0 = 1.2345 # Frecuencia fundamental
t = np.arange(-10, 10, step=1/Fs)
s = np.cos(2*np.pi*f0*t)
p = []
for T in Ts:
    w = np.zeros_like(t)
    w[np.absolute(t) < T/2] = 1
    sT = s*w
    f = sfft.rfftfreq(n=len(sT), d=1/Fs)
    ST = np.absolute(sfft.rfft(sT))
    pp = [Figure(plot_width=300, plot_height=150, toolbar_location="below"),
          Figure(plot_width=300, plot_height=150, toolbar_location="below")]
    pp[0].line(t, sT, line_width=3, alpha=0.75)
    pp[1].line(f, ST, line_width=3, alpha=0.75, legend_label=f"T={T}")
    p.append(pp)
    
p[-1][0].xaxis[0].axis_label = 'Tiempo [s]'
p[-1][1].xaxis[0].axis_label = 'Frecuencia [Hz]'
show(gridplot(p))

## Fuga espectral

La aparición de los lóbulos laterales por truncamiento de la señal es un efecto que se conoce como **fuga espectral** o *spectral leak*

La fuga espectral es una redistribución de la energía de un cierto componente espectral hacia sus frecuencias vecinas debido a las **discontinuidades o quiebres** en la periodicidad de la señal en sus bordes

Observe a continuación la diferencia entre el espectro de una señal donde su periodicidad calza perfecto con la ventana (fila superior) versus cuando esto no ocurre, que sería el caso más general (fila inferior)

In [ ]:
import scipy.fft as sfft

T = 2 # Duración de la ventana
Fs = 20 # Frecuencia de muestre
f0s = [2.0, 2.15] # Frecuencia fundamental
p = []
for f0 in f0s:
    t = np.arange(-T, T, step=1/Fs)
    sT = np.cos(2*np.pi*f0*t)
    f = sfft.rfftfreq(n=len(sT), d=1/Fs)
    ST = np.absolute(sfft.rfft(sT))
    pp = [Figure(plot_width=300, plot_height=150, toolbar_location="below"),
          Figure(plot_width=300, plot_height=150, toolbar_location="below", y_range=(-1, 41))]
    pp[0].line(t, sT, line_width=3, alpha=0.75)
    pp[1].line(f, ST, line_width=3, alpha=0.75)
    p.append(pp)
    
p[-1][0].xaxis[0].axis_label = 'Tiempo [s]'
p[-1][1].xaxis[0].axis_label = 'Frecuencia [Hz]'
show(gridplot(p))

La fuga espectral es indeseada ya que no aporta información real

Podemos eliminar este efecto si hacemos que los bordes de la señal "calcen" antes de calcular su espectro

Este proceso de suavizado de bordes se llama **enventanado**


## Enventanado espectral

Es el proceso de multiplicar la señal por una ventana para alterar las características de su espectro

Lamentablemente no es posible eliminar la fuga espectral completamente con enventanado pero si podemos modificar como se redistribuye la energía

Existen muchas ventanas y cada una representa un compromiso (*trade-off*) distinto entre la **concentración o ancho del lóbulo principal y la atenuación de los lóbulos laterales**

En general se tiene que

- Mientras más abrupta es la ventana, más concentrado será el lóbulo principal y más fuertes serán los lóbulos laterales
- Mientras más suave sea la ventana, menos concentrado será el lóbulo principal y más débiles serán los lóbulos laterales

Es decir que siempre estamos sacrificando resolución por limpieza

Revisemos a continuación algunas ventanas (columna de la izquierda), su efecto al multiplicar con la señal (columna central) y en el espectro resultante (columna derecha)


In [ ]:
import scipy.fft as sfft
from scipy.signal import cosine, blackman, tukey

T = 2 # Duración de la ventana
Fs = 20 # Frecuencia de muestre
f0 = 2.15 # Frecuencia fundamental
t = np.arange(-T, T, step=1/Fs)
sT = np.cos(2*np.pi*f0*t)
p = []
    
for wname in ['rect', 'tukey', 'cosine', 'blackman']:
    if wname == 'rect':
        w = np.ones_like(sT)
    elif wname == 'tukey':
        w = tukey(len(sT))
    elif wname == 'cosine':
        w = cosine(len(sT))
    elif wname == 'blackman':
        w = blackman(len(sT))
    
    f = sfft.rfftfreq(n=len(sT), d=1/Fs)
    ST = np.absolute(sfft.rfft(sT*w))
    f_long = sfft.rfftfreq(n=len(sT)*10, d=1/Fs)
    ST_long = np.absolute(sfft.rfft(sT*w, n=len(sT)*10))
    pp = [Figure(plot_width=300, plot_height=150, toolbar_location="below", y_range=(-0.1, 1.1)),
          Figure(plot_width=300, plot_height=150, toolbar_location="below"),
          Figure(plot_width=300, plot_height=150, toolbar_location="below", y_range=(-1, 35))]
    pp[0].line(t, w, line_width=3, alpha=0.75)
    pp[1].line(t, sT*w, line_width=3, alpha=0.75)
    pp[2].line(f_long, ST_long, line_width=3, alpha=0.75, legend_label=wname)
    pp[2].scatter(f, ST, color='black', line_width=3, alpha=0.75)
    p.append(pp)
    
p[-1][0].xaxis[0].axis_label = 'Tiempo [s]'
p[-1][1].xaxis[0].axis_label = 'Tiempo [s]'
p[-1][2].xaxis[0].axis_label = 'Frecuencia [Hz]'
show(gridplot(p))

Las ventanas espectrales más utilizadas están implementadas en el módulo [`scipy.signal`](https://docs.scipy.org/doc/scipy/reference/signal.windows.html?highlight=window#module-scipy.signal.windows)

En el ejemplo anterior usamos la ventana 

- Rectangular: Borde con discontinuidad fuerte, es la que tiene más concentración de lóbulo principal pero también más lóbulos laterales
- [Tukey](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.windows.tukey.html#scipy.signal.windows.tukey) con parámetro $\alpha=0.5$, es una transición entre la ventana rectangular y la ventana coseno
- [Cosine](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.windows.cosine.html#scipy.signal.windows.cosine), como su nombre lo indica tiene forma de coseno en $[-\pi/2, \pi/2]$
- [Blackman](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.windows.blackman.html#scipy.signal.windows.blackman): Borde muy suave, es la que tiene menos concentración de lóbulo principal pero también menos lóbulos laterales


Otra ventana interesante es la ["ventana de Kaiser"](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.windows.kaiser.html#scipy.signal.windows.kaiser) cuyo parámetro $\beta$ le permite asemejarse a otros ventanas de forma similar a la ventana de Tukey pero más flexible

- $\beta=0$: ventana rectangular
- $\beta=5$: similar a ventana coseno
- $\beta=8.8$ similar a la ventana de Blackman

Ejemplo a continuación

In [ ]:
import scipy.fft as sfft
from scipy.signal import kaiser

T = 2 # Duración de la ventana
Fs = 20 # Frecuencia de muestre
f0 = 2.15 # Frecuencia fundamental
t = np.arange(-T, T, step=1/Fs)
sT = np.cos(2*np.pi*f0*t)
p = []
    
for beta in [0, 5, 8.8]:
    w = kaiser(len(sT), beta=beta)
    f = sfft.rfftfreq(n=len(sT), d=1/Fs)
    ST = np.absolute(sfft.rfft(sT*w))
    f_long = sfft.rfftfreq(n=len(sT)*10, d=1/Fs)
    ST_long = np.absolute(sfft.rfft(sT*w, n=len(sT)*10))
    pp = [Figure(plot_width=300, plot_height=150, toolbar_location="below", y_range=(-0.1, 1.1)),
          Figure(plot_width=300, plot_height=150, toolbar_location="below"),
          Figure(plot_width=300, plot_height=150, toolbar_location="below", y_range=(-1, 35))]
    pp[0].line(t, w, line_width=3, alpha=0.75)
    pp[1].line(t, sT*w, line_width=3, alpha=0.75)
    pp[2].line(f_long, ST_long, line_width=3, alpha=0.75, legend_label="beta: "+str(beta))
    pp[2].scatter(f, ST, color='black', line_width=3, alpha=0.75)
    p.append(pp)
    
p[-1][0].xaxis[0].axis_label = 'Tiempo [s]'
p[-1][1].xaxis[0].axis_label = 'Tiempo [s]'
p[-1][2].xaxis[0].axis_label = 'Frecuencia [Hz]'
show(gridplot(p))

### ¿Cómo escoger que ventana usar?

Esta es una pregunta sumamente importante pero difícil de responder ya que depende de cada problema

A continuación algunas indicaciones

- Para propósito general o como primera aproximación usar una ventana que entregue un buen compromiso, por ejemplo Cosine, Hamming o Hanning
- Si se requiere separar/discriminar frecuencias muy cercanas conviene usar una ventana que resalte más el lóbulo principal, por ejemplo rectangular o Tukey
- Si las frecuencias de interés están muy separadas entonces conviene usar una ventana que suprima más fuertemente los lóbulos laterales como la de blackman

